# Deploy

Deploy `basic_solana` example on mainnet.

In [15]:
!dfx identity use admin

Using identity: "admin".


In [16]:
!dfx deploy --ic basic_solana

Deploying: basic_solana
All canisters have already been created.
Building canister 'basic_solana'.                                        
Executing 'cargo build --no-default-features --target wasm32-unknown-unknown --release'
ilding canisters...                                                            Compiling sol_rpc_client v0.2.0 (/Users/greg/git/sol-rpc-canister/libs/client)
   Compiling basic_solana v0.2.0 (/Users/greg/git/sol-rpc-canister/examples/basic_solana)
    Finished `release` profile [optimized] target(s) in 12.26sn)           
⠒ Installing canisters...                                                       ⠁ Installing canisters...                                                       
⠒ Installing canisters...ic_solana, with canister ID vond4-2qaaa-aaaar-qbmiq-cai                                                                        
⠂ Installing canisters...ic_solana, with canister ID vond4-2qaaa-aaaar-qbmiq-cai                                                       

In [17]:
!dfx canister status --ic --all


Canister status call result for basic_solana.
Status: Running
Controllers: 4nct6-qlari-qhlgv-qkhdd-5hak6-gtdms-ra2ya-gi7qq-2lref-zg34x-bqe ezu3d-2mifu-k3bh4-oqhrj-mbrql-5p67r-pp6pr-dbfra-unkx5-sxdtv-rae mf7xa-laaaa-aaaar-qaaaa-cai
Memory allocation: 0 Bytes
Compute allocation: 0 %
Freezing threshold: 2_592_000 Seconds
Idle cycles burned per day: 67_686_185 Cycles
Memory Size: 2_532_485 Bytes
Balance: 6_950_090_348_934 Cycles
Reserved: 0 Cycles
Reserved cycles limit: 5_000_000_000_000 Cycles
Wasm memory limit: 3_221_225_472 Bytes
Wasm memory threshold: 0 Bytes
Module hash: 0x4ae0c236e7b5bec153f527c6b291088af48e202d5eb154f8d9423e0005bcf160
Number of queries: 0
Instructions spent in queries: 0
Total query request payload size: 0 Bytes
Total query response payload size: 0 Bytes
Log visibility: controllers
